In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pickle
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.utils import shuffle
import random as rand
from nltk import word_tokenize

import keras.backend as K
import tensorflow as tf

from keras.models import Model
from keras.layers import Input, Dense, Lambda, Dropout
from keras.utils import to_categorical

Using TensorFlow backend.


# load OANC corpus

In [35]:
df = pd.read_pickle('data/discourse_markers/oanc_df.zip')

In [36]:
df.head()

,text,label,sents,clean_and_tokenized,vectors
0,In my three decades of teaching university cou...,non-fiction/OUP/Berk/ch1,[In my three decades of teaching university co...,"[[In, my, three, decades, of, teaching, univer...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, ..."
1,"As a byproduct of those experiences, parents r...",non-fiction/OUP/Berk/ch1,"[As a byproduct of those experiences, parents ...","[[As, a, byproduct, of, those, experiences, ,,...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, ..."
2,"•Bob and Sharon, parents of a 4-year-old: Our ...",non-fiction/OUP/Berk/ch1,"[When we looked for a preschool, many programs...","[[When, we, looked, for, a, preschool, ,, many...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, ..."
3,"•Angela, mother of a 4-year-old and 6-year-old...",non-fiction/OUP/Berk/ch1,[I’ve read that it’s the quality of time we sp...,"[[I, ’, ve, read, that, it, ’, s, the, quality...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, ..."
4,"•Talia, mother of a 7-year-old: My son Anselmo...",non-fiction/OUP/Berk/ch1,[His father ﬁrmly insists that he do it by him...,"[[His, father, ﬁrmly, insists, that, he, do, i...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, ..."


# train doc2vec encodings

In [42]:
X_tokens = []
for idx, row in tqdm(df.iterrows(), total=len(df)):
    for item in row['clean_and_tokenized']:
        X_tokens.append(item)

In [43]:
tagged = []
for i, sent in enumerate(tqdm(X_tokens)):
    tagged.append(TaggedDocument(words = sent, tags = [str(i)]))

In [45]:
d2v_oanc = Doc2Vec(vector_size = 50, min_count = 1, dm = 1)
d2v_oanc.build_vocab(tagged)
print('vocabulary built')

vocabulary built


In [49]:
d2v_oanc.train(tagged, total_examples = d2v_oanc.corpus_count, epochs = 20)
print('training finished')
d2v_oanc.save("data/discourse_markers/d2v_oanc.model")
print("trained & saved")

training finished
trained & saved


# vectorize texts and add to new df

In [ ]:
vecs = []

for idx, row in tqdm(brown_disc_df.iterrows(), total = len(brown_disc_df)):
    vecs.append(d2v.docvecs[str(idx)])
        
brown_disc_df['vec'] = vecs